<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

total no of sub arrays which sum up to k.

In [ ]:
import java.util.ArrayList;
import java.util.HashMap;

public class totalSubarrayEqualToK {
    public static void main(String[] args){
        int[] arr = {1,-1,0};
        int k = 0;
        int count = 0;
        HashMap<Integer, Integer> prefixSumCount = new HashMap<>();
        int sum = 0;
        prefixSumCount.put(0, 1);
        for (int i = 0; i < arr.length; i++) {
            sum += arr[i];
            if (prefixSumCount.containsKey(sum - k)) {
                count += prefixSumCount.get(sum - k);
            }
            prefixSumCount.put(sum, prefixSumCount.getOrDefault(sum, 0) + 1);
        }
        System.out.println(count);
    }
}



Binary search

In [ ]:
public class binarySearch {
    public static void main(String[] args){
        int[] nums = {-1,0,3,5,9,12};
        int target = 9;
        int left = 0;
        int right = nums.length-1;
        while(left <= right){
            int mid = (left+right)/2;
            if(nums[mid] == target){
                System.out.println("Element found at index: "+mid);
                break;
            }
            else if(nums[mid]<target){
                left = mid +1;
            }
            else{
                right = mid-1;
            }
        }
        System.out.println("Element no found");
    }

}


floor value


In [ ]:
public class floorOfK {
    public static void main(String[] args) {
        int[] nums = {1,2,4,10,11,12,19};
        int left = 0;
        int right = nums.length-1;
        int target = 5;
        int floorindex =-1;
        while(left<=right){
            int mid = (left + right)/2;
            if(nums[mid]==target){
                System.out.println(mid);
                break;
            } else if (nums[mid]<target) {
                floorindex = mid;
                left = mid +1;
            }
            else{
                right = mid -1;
            }
        }
        System.out.println(floorindex);
    }
}


ceil and floor value

In [ ]:
import java.util.Arrays;

public class floorAndCielValueOFK {
    public static void main(String[] args) {
        int[] nums = {5, 6, 8, 9, 6, 5, 5, 6};
        Arrays.sort(nums);
        int left = 0;
        int right = nums.length-1;
        int target = 7;
        int floorIndex =-1;
        int ceilIndex = -1;
        while(left<=right){
            int mid = (left + right)/2;
            if(nums[mid]==target){
                System.out.println(mid);
                break;
            } else if (nums[mid]<target) {
                floorIndex = nums[mid];
                left = mid +1;
            }
            else{
                ceilIndex = nums[mid];
                right = mid -1;
            }
        }
        System.out.println(floorIndex+" "+ceilIndex);
    }
}


Html code

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="navbar">
        <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqV8qU3m3W7cqHR9zkcdtry3bP-rQGybA-sw&s"
        alt="Logo" id="image">
        <h1 class="mainHeading">Expenses Calculator!</h1>
    </div>
    <div class="expensesForm">
        <h3 class="heading">Expenses Form</h3>
        <form class="formElements" onsubmit="getdetails(event)">
           <div class="select">
                <label for="expenseName">Expense Name: </label>
                <input type="text" id="expenseName" name="expenseName" required>
           </div>
           <div class="select">
                <label for="options">Select a category:</label>
                <select id="options">
                    <option value="Select a Category">Select a Category</option>
                    <option value="Dressing">Dressing</option>
                    <option value="Food">Food</option>
                    <option value="Travel">Travel</option>
                    <option value="Other">Other</option>
                </select>
            </div>
            <div class="select" id="otherInputContainer"></div>
            <div class="select">
                <label for="totalAmount">Total Amount: </label>
                <input type="number" id="totalAmount" name="totalAmount" required>
            </div>
            <div class="select">
                <button type="submit">Submit</button>
            </div>
        </form>
    </div>
    <div class="additionalDetails">
        <div class="filtering">filtering</div>
        <div class="categoryWiseExpenses">Expenses</div>
    </div>
    <script src="actions.js"></script>
</body>
</html>

css code

In [ ]:
body{
    display: flex;
    flex-direction: column;
}

.navbar{
    height: 20%;
    display: flex;
    border-bottom: 2px solid black;
    /* padding: 1%; */
    justify-content: flex-start;
    align-items: center;
}

#image{
    height: 50px;
    width: 100px;
    border-radius: 5%;
    padding-right: 2%;

}

.heading{
    margin: 0;
    padding-bottom: 1%;
}

.expensesForm{
    height: 30%;
    padding: 1%;
    border-bottom: 2px solid black;
}

.formElements{
    display: flex;
    justify-content: space-between;
}
.select{
    display: flex;
    /* justify-content: center; */
}

#otherinput{
    height: 5px;
    border-width: 1px;
    border-style: solid;
    border-color: light-dark(rgb(118, 118, 118)),
}

.additionalDetails{
    display: flex;
}

.filtering{
    width: 50%;
    border-right: 2px solid black;
}

.categoryWiseExpenses{
    width: 50%;
    border-right: 2px solid black;
}


javascript

In [ ]:
// This is to take only aphabelts in expensename element
let expenseNameInput = document.getElementById("expenseName");
expenseNameInput.addEventListener("input", function(event) {
            // Remove any non-alphabet characters
    event.target.value = event.target.value.replace(/[^A-Za-z ]+/g, '');
});


let options = document.getElementById("options");
let newele;
let newlabel;
options.onchange = () => {
    const container = document.getElementById("otherInputContainer");
    // Clear the container before adding new elements
    container.innerHTML = '';
    if (options.value === "Other") {
        // Create and append a label
        newlabel = document.createElement("label");
        newlabel.textContent = "Other Category:";
        container.appendChild(newlabel);
        // Create and append a textarea
        newele = document.createElement("textarea");
        newele.setAttribute("id","otherinput")
        // This is to take only aphabelts in otherinput textarea element
        container.addEventListener("input", function(event) {
            // Remove any non-alphabet characters
        event.target.value = event.target.value.replace(/[^A-Za-z ]+/g, '');
        });
        container.appendChild(newele);
    }
};

function getdetails(event){
    // event.preventDefault()
    let category
    if(document.getElementById("options").value === "Other"){
        category = document.getElementById("otherInputContainer").querySelector("textarea").value
    }
    else{
        category = document.getElementById("options").value
    }
    console.log(category)
    let details = {
        expenseName: document.getElementById("expenseName").value,
        category: category,
        totalAmount: document.getElementById("totalAmount").value
    };
    console.log(details)
    localStorage.setItem("detail", JSON.stringify(details));
}




